# DL_mini project #3

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)


In [2]:
tf.__version__

'2.8.0'

In [3]:
import pandas as pd
kc_house = pd.read_csv('../../DL/data/kc_house_data.csv')

In [9]:
from sklearn.utils import shuffle
kc_house = shuffle(kc_house, random_state = 42)

In [10]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer()

x=["date","price","bedrooms","bathrooms","sqft_living","sqft_lot","floors","waterfront","view","condition","grade","sqft_above","sqft_basement","yr_built","yr_renovated","zipcode","lat","long","sqft_living15","sqft_lot15"]

df = pd.DataFrame(kc_house, columns=x)
df['year'] = pd.DatetimeIndex(df['date']).year
df = df.drop(['date'], axis=1)

df = pd.DataFrame(imputer.fit_transform(df))
df.columns = ["price","bedrooms","bathrooms","sqft_living","sqft_lot","floors","waterfront","view","condition","grade","sqft_above","sqft_basement","yr_built","yr_renovated","zipcode","lat","long","sqft_living15","sqft_lot15","year"]

df.describe()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year
count,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652,2014.322954
std,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631,0.467616
min,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000,2014.000000
25%,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000,2014.000000
50%,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000,2014.000000
75%,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000,2015.000000
max,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000,2015.000000


In [11]:
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year
0,365000.0,4.0,2.25,2070.0,8893.0,2.0,0.0,0.0,4.0,8.0,2070.0,0.0,1986.0,0.0,98058.0,47.4388,-122.162,2390.0,7700.0,2014.0
1,865000.0,5.0,3.00,2900.0,6730.0,1.0,0.0,0.0,5.0,8.0,1830.0,1070.0,1977.0,0.0,98115.0,47.6784,-122.285,2370.0,6283.0,2014.0
2,1038000.0,4.0,2.50,3770.0,10893.0,2.0,0.0,2.0,3.0,11.0,3770.0,0.0,1997.0,0.0,98006.0,47.5646,-122.129,3710.0,9685.0,2014.0
3,1490000.0,3.0,3.50,4560.0,14608.0,2.0,0.0,2.0,3.0,12.0,4560.0,0.0,1990.0,0.0,98034.0,47.6995,-122.228,4050.0,14226.0,2015.0
4,711000.0,3.0,2.50,2550.0,5376.0,2.0,0.0,0.0,3.0,9.0,2550.0,0.0,2004.0,0.0,98052.0,47.6647,-122.083,2250.0,4050.0,2014.0


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          21613 non-null  float64
 1   bedrooms       21613 non-null  float64
 2   bathrooms      21613 non-null  float64
 3   sqft_living    21613 non-null  float64
 4   sqft_lot       21613 non-null  float64
 5   floors         21613 non-null  float64
 6   waterfront     21613 non-null  float64
 7   view           21613 non-null  float64
 8   condition      21613 non-null  float64
 9   grade          21613 non-null  float64
 10  sqft_above     21613 non-null  float64
 11  sqft_basement  21613 non-null  float64
 12  yr_built       21613 non-null  float64
 13  yr_renovated   21613 non-null  float64
 14  zipcode        21613 non-null  float64
 15  lat            21613 non-null  float64
 16  long           21613 non-null  float64
 17  sqft_living15  21613 non-null  float64
 18  sqft_l

In [13]:
df.describe()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year
count,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652,2014.322954
std,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631,0.467616
min,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000,2014.000000
25%,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000,2014.000000
50%,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000,2014.000000
75%,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000,2015.000000
max,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000,2015.000000


In [14]:
X_train_full = df.copy()

In [15]:
X_train_full

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year
0,365000.0,4.0,2.25,2070.0,8893.0,2.0,0.0,0.0,4.0,8.0,2070.0,0.0,1986.0,0.0,98058.0,47.4388,-122.162,2390.0,7700.0,2014.0
1,865000.0,5.0,3.00,2900.0,6730.0,1.0,0.0,0.0,5.0,8.0,1830.0,1070.0,1977.0,0.0,98115.0,47.6784,-122.285,2370.0,6283.0,2014.0
2,1038000.0,4.0,2.50,3770.0,10893.0,2.0,0.0,2.0,3.0,11.0,3770.0,0.0,1997.0,0.0,98006.0,47.5646,-122.129,3710.0,9685.0,2014.0
3,1490000.0,3.0,3.50,4560.0,14608.0,2.0,0.0,2.0,3.0,12.0,4560.0,0.0,1990.0,0.0,98034.0,47.6995,-122.228,4050.0,14226.0,2015.0
4,711000.0,3.0,2.50,2550.0,5376.0,2.0,0.0,0.0,3.0,9.0,2550.0,0.0,2004.0,0.0,98052.0,47.6647,-122.083,2250.0,4050.0,2014.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,378000.0,3.0,1.50,1000.0,6914.0,1.0,0.0,0.0,3.0,7.0,1000.0,0.0,1947.0,0.0,98125.0,47.7144,-122.319,1000.0,6947.0,2014.0
21609,399950.0,3.0,2.50,3087.0,5002.0,2.0,0.0,0.0,3.0,8.0,3087.0,0.0,2014.0,0.0,98023.0,47.2974,-122.349,2927.0,5183.0,2014.0
21610,575000.0,3.0,2.50,2120.0,4780.0,2.0,0.0,0.0,3.0,7.0,2120.0,0.0,2004.0,0.0,98053.0,47.6810,-122.032,1690.0,2650.0,2014.0
21611,245000.0,1.0,0.75,380.0,15000.0,1.0,0.0,0.0,3.0,5.0,380.0,0.0,1963.0,0.0,98168.0,47.4810,-122.323,1170.0,15000.0,2014.0


In [16]:
y_train = X_train_full[['price']]
X_train_full = X_train_full.drop(['price'],axis=1)

In [17]:
X_train_full

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year
0,4.0,2.25,2070.0,8893.0,2.0,0.0,0.0,4.0,8.0,2070.0,0.0,1986.0,0.0,98058.0,47.4388,-122.162,2390.0,7700.0,2014.0
1,5.0,3.00,2900.0,6730.0,1.0,0.0,0.0,5.0,8.0,1830.0,1070.0,1977.0,0.0,98115.0,47.6784,-122.285,2370.0,6283.0,2014.0
2,4.0,2.50,3770.0,10893.0,2.0,0.0,2.0,3.0,11.0,3770.0,0.0,1997.0,0.0,98006.0,47.5646,-122.129,3710.0,9685.0,2014.0
3,3.0,3.50,4560.0,14608.0,2.0,0.0,2.0,3.0,12.0,4560.0,0.0,1990.0,0.0,98034.0,47.6995,-122.228,4050.0,14226.0,2015.0
4,3.0,2.50,2550.0,5376.0,2.0,0.0,0.0,3.0,9.0,2550.0,0.0,2004.0,0.0,98052.0,47.6647,-122.083,2250.0,4050.0,2014.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,3.0,1.50,1000.0,6914.0,1.0,0.0,0.0,3.0,7.0,1000.0,0.0,1947.0,0.0,98125.0,47.7144,-122.319,1000.0,6947.0,2014.0
21609,3.0,2.50,3087.0,5002.0,2.0,0.0,0.0,3.0,8.0,3087.0,0.0,2014.0,0.0,98023.0,47.2974,-122.349,2927.0,5183.0,2014.0
21610,3.0,2.50,2120.0,4780.0,2.0,0.0,0.0,3.0,7.0,2120.0,0.0,2004.0,0.0,98053.0,47.6810,-122.032,1690.0,2650.0,2014.0
21611,1.0,0.75,380.0,15000.0,1.0,0.0,0.0,3.0,5.0,380.0,0.0,1963.0,0.0,98168.0,47.4810,-122.323,1170.0,15000.0,2014.0


In [18]:
X_train = X_train_full.copy()
X_train

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year
0,4.0,2.25,2070.0,8893.0,2.0,0.0,0.0,4.0,8.0,2070.0,0.0,1986.0,0.0,98058.0,47.4388,-122.162,2390.0,7700.0,2014.0
1,5.0,3.00,2900.0,6730.0,1.0,0.0,0.0,5.0,8.0,1830.0,1070.0,1977.0,0.0,98115.0,47.6784,-122.285,2370.0,6283.0,2014.0
2,4.0,2.50,3770.0,10893.0,2.0,0.0,2.0,3.0,11.0,3770.0,0.0,1997.0,0.0,98006.0,47.5646,-122.129,3710.0,9685.0,2014.0
3,3.0,3.50,4560.0,14608.0,2.0,0.0,2.0,3.0,12.0,4560.0,0.0,1990.0,0.0,98034.0,47.6995,-122.228,4050.0,14226.0,2015.0
4,3.0,2.50,2550.0,5376.0,2.0,0.0,0.0,3.0,9.0,2550.0,0.0,2004.0,0.0,98052.0,47.6647,-122.083,2250.0,4050.0,2014.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,3.0,1.50,1000.0,6914.0,1.0,0.0,0.0,3.0,7.0,1000.0,0.0,1947.0,0.0,98125.0,47.7144,-122.319,1000.0,6947.0,2014.0
21609,3.0,2.50,3087.0,5002.0,2.0,0.0,0.0,3.0,8.0,3087.0,0.0,2014.0,0.0,98023.0,47.2974,-122.349,2927.0,5183.0,2014.0
21610,3.0,2.50,2120.0,4780.0,2.0,0.0,0.0,3.0,7.0,2120.0,0.0,2004.0,0.0,98053.0,47.6810,-122.032,1690.0,2650.0,2014.0
21611,1.0,0.75,380.0,15000.0,1.0,0.0,0.0,3.0,5.0,380.0,0.0,1963.0,0.0,98168.0,47.4810,-122.323,1170.0,15000.0,2014.0


In [19]:
y_train

,price
0,365000.0
1,865000.0
2,1038000.0
3,1490000.0
4,711000.0
...,...
21608,378000.0
21609,399950.0
21610,575000.0
21611,245000.0


In [20]:
price_mean = y_train.mean(axis=0)
print(price_mean)

price    540088.141767
dtype: float64


In [21]:
mean = X_train.mean(axis=0)
X_train -= mean
std = X_train.std(axis=0)
X_train /= std

In [22]:
X_train

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year
0,0.676469,0.175603,-0.010779,-0.150022,0.936484,-0.087171,-0.305752,0.907533,0.291909,0.340070,-0.658666,0.510491,-0.210124,-0.372672,-0.875067,0.368508,0.588638,-0.185629,-0.690639
1,1.751667,1.149422,0.892927,-0.202242,-0.915406,-0.087171,-0.305752,2.444237,0.291909,0.050247,1.759003,0.204092,-0.210124,0.692649,0.854102,-0.504895,0.559458,-0.237526,-0.690639
2,0.676469,0.500209,1.840184,-0.101736,0.936484,-0.087171,2.304132,-0.629172,2.844104,2.392985,-0.658666,0.884979,-0.210124,-1.344543,0.032819,0.602836,2.514545,-0.112930,-0.690639
3,-0.398728,1.798635,2.700337,-0.012046,0.936484,-0.087171,2.304132,-0.629172,3.694836,3.346986,-0.658666,0.646669,-0.210124,-0.821228,1.006378,-0.100147,3.010612,0.053382,1.447868
4,-0.398728,0.500209,0.511846,-0.234931,0.936484,-0.087171,-0.305752,-0.629172,1.142641,0.919717,-0.658666,1.123290,-0.210124,-0.484811,0.755230,0.929475,0.384375,-0.319308,-0.690639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,-0.398728,-0.798217,-1.175797,-0.197800,-0.915406,-0.087171,-0.305752,-0.629172,-0.558823,-0.952058,-0.658666,-0.817240,-0.210124,0.879547,1.113910,-0.746324,-1.439400,-0.213207,-0.690639
21609,-0.398728,0.500209,1.096532,-0.243960,0.936484,-0.087171,-0.305752,-0.629172,0.291909,1.568196,-0.658666,1.463734,-0.210124,-1.026816,-1.895536,-0.959349,1.372132,-0.277813,-0.690639
21610,-0.398728,0.500209,0.043661,-0.249320,0.936484,-0.087171,-0.305752,-0.629172,-0.558823,0.400450,-0.658666,1.123290,-0.210124,-0.466121,0.872865,1.291618,-0.432676,-0.370583,-0.690639
21611,-2.549123,-1.772037,-1.850854,-0.002582,-0.915406,-0.087171,-0.305752,-0.629172,-2.260286,-1.700768,-0.658666,-0.272530,-0.210124,1.683210,-0.570514,-0.774728,-1.191367,0.081729,-0.690639


In [25]:
def build_model():
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape = (X_train.shape[1],)))
    #model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(300, kernel_initializer = 'lecun_normal', activation = "selu"))
    #model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(300, kernel_initializer = 'lecun_normal', activation = "selu"))
    #model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(300, kernel_initializer = 'lecun_normal', activation = "selu"))
    #model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(1))
    
    model.compile(loss="mse", 
                  optimizer = "adam", 
                  metrics=["mse"])
    return model

In [23]:
def build_model(): 
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=(X_train.shape[1],)))
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(1))
    
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mse'])
    
    return model

In [26]:
k = 10
num_val = len(X_train) // k
num_epochs = 100
all_scores = []
temp = []

for i in range(k):
    print('processing fold #', i)
    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)
    
    model = build_model()
    model.fit(X_train_part, y_train_part,
              epochs=num_epochs, 
              verbose=0)

    val_mse, val_mae = model.evaluate(X_val, y_val, verbose=0)
    all_scores.append(val_mse)
    temp.append(val_mse)    

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
processing fold # 5
processing fold # 6
processing fold # 7
processing fold # 8
processing fold # 9


In [27]:
all_scores

[16869612544.0,
 14947997696.0,
 11810857984.0,
 13952116736.0,
 14038784000.0,
 11409452032.0,
 12698981376.0,
 11545284608.0,
 14699189248.0,
 11849213952.0]

In [28]:
np.mean(all_scores)

13382149017.6